In this workbook, I apply the Periodic Kernel to test case 19, variation 1, 3, 5, 7, 8. 

In [1]:
cd ../

/home/riddhisw/Documents/2017/Scripts_Git


In [2]:
import GPy, numpy as np 
from matplotlib import pyplot as plt
from gprtf.common import get_data

The optimisation problem over all 4 parameters for GPR is difficult. We link the periodicity and the lengthscale to the longest and shortest timescales of the system during training.

Namely:
    - p = N_train
    - l = 3 * Delta T 
    
Sigma (kernel / process noise variance) and R (Kalman msmt noise variance) orders were estimated from optimisation during Kalman filtering. 

We note that if the optimal periodicity >> N_train or N, the algorithm tends to pick l <<< Delta T, and a failure results.

We recreate this failure by constraining periodicity a small distance away from n_train.

Test Case 7, 1 (Four Fourier Components) and Test Case 19, 1 (Severely Undersampled 'White' Noise with 1% SNR) with the following parameters:

    - n_train = 2000
    - Delta_T = 0.001
    - sigma = 1.0
    - R =  1.0
    - SigmaMax = 1000.0 
    - Rmax = 30.0

We then confirm that the procedure to narrow the optimisation problem (as above) works by:

    - Case A: unconstrained Periodicity
    - Case B: m1.std_periodic.period.constrain_bounded(1970, 1971)
    - Case C: m1.std_periodic.period.constrain_bounded(2075, 2076)

In [3]:
test_case = 19
variation = 8
n_train = 2000
Delta_T = 0.001
sigma = 1.0
R =  1.0
SigmaMax = 10. #20.0 # tc 7 var 1, sigma = 1000.0
Rmax = 15. #15.0
# sigma and periodicity move together.
# unconstrained == periodicity --> infinity, sigma --> large, predictions -->zero
# failure = increasing periodicity (algorithm appears to be predicting zero)

In [4]:
def trainingdata(test_case, variation):
    x, y, testx, truth, n_predict= get_data(test_case, variation, 
                                        randomize='y')
    num = truth.shape[0]
    X = x[0:n_train,np.newaxis]
    Y = y[0:n_train,np.newaxis]
    return X, Y, testx

period = 2000.0

In [5]:
pred_list = []

for idx_runs in xrange(5):
    X, Y, testx = trainingdata(test_case, variation)
    print 'Run', idx_runs
    kern5 = GPy.kern.StdPeriodic(1, period=period, variance=sigma, lengthscale=3*Delta_T )
    gauss = GPy.likelihoods.Gaussian(variance=R)
    exact = GPy.inference.latent_function_inference.ExactGaussianInference()
    m1 = GPy.core.GP(X=X, Y=Y, kernel=kern5, likelihood=gauss, inference_method=exact)
    m1.std_periodic.variance.constrain_bounded(0, SigmaMax)
    m1.Gaussian_noise.variance.constrain_bounded(0, Rmax)
    print 'Before Optimisation: ', m1.Gaussian_noise.variance[0], m1.std_periodic.variance[0], m1.std_periodic.period[0], m1.std_periodic.lengthscale[0]
    m1.optimize(messages=1, ipython_notebook=True)
    print 'After Optimisation: ', m1.Gaussian_noise.variance[0], m1.std_periodic.variance[0], m1.std_periodic.period[0], m1.std_periodic.lengthscale[0]
    pred_list.append(m1.predict(testx[:,np.newaxis])[0].flatten())

 gprtf/common.py:32: VisibleDeprecationWarning:using a non-integer number instead of an integer will result in an error in the future
 gprtf/common.py:33: VisibleDeprecationWarning:using a non-integer number instead of an integer will result in an error in the future
 /home/riddhisw/anaconda2/lib/python2.7/site-packages/paramz/transformations.py:109: RuntimeWarning:overflow encountered in expm1


('Shapes', 200, 200, (150,))
Run 0
Before Optimisation:  1.0 1.0 2000.0 0.003
After Optimisation:  14.9999999571 9.99999995901 1990.71135267 0.00865596179682
('Shapes', 200, 200, (150,))
Run 1
Before Optimisation:  1.0 1.0 2000.0 0.003
After Optimisation:  15.0 10.0 2023.57935597 0.00845528834906
('Shapes', 200, 200, (150,))
Run 2
Before Optimisation:  1.0 1.0 2000.0 0.003
After Optimisation:  15.0 9.99999999998 1984.89301391 0.00612705130708
('Shapes', 200, 200, (150,))
Run 3
Before Optimisation:  1.0 1.0 2000.0 0.003
After Optimisation:  14.9999999968 9.99999999807 1979.35822983 0.00962085214274
('Shapes', 200, 200, (150,))
Run 4
Before Optimisation:  1.0 1.0 2000.0 0.003
After Optimisation:  14.9999997142 9.99999979468 1999.99936797 0.00847054482999


In [6]:
style = ['b', 'g', 'c', 'brown', 'purple']
x, y, testx, truth, n_predict= get_data(test_case, variation, 
                                        randomize='n')

('Shapes', 2000, 2000, (150,))


In [7]:
plt.figure(figsize=(10,5))
for idx_pred in xrange(5):
    plt.plot(testx, pred_list[idx_pred], style[idx_pred], alpha=0.5, label='Predictions')
plt.plot(testx, truth[2000 -50 : ], 'r', label='Truth')
plt.legend(loc=0)
plt.title('GPy PRE, Testcase %s Var %s RMax %s SigMax %s' %(test_case, variation, Rmax, SigmaMax))
plt.savefig('Gpy_PER_try2_tc_'+str(test_case)+'_var_'+str(variation))
plt.close()